Start by installing all requirements

In [1]:
!pip install -r /kaggle/input/requirements/requirements.txt
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=39f3297d6f7eff929337fcb07c682c845690e32ecfdf0a542e52f2716c126ff1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e269c1446a2bc2bfbc1483d4497863936f47fc88c9a29cb4314b53169d3dd3b8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built langdetect seqeval


Next create a function for reading in csvs

In [2]:
import pandas as pd

# read in train, dev, test data
train_data = pd.read_csv("/kaggle/input/credibility-data/full_data_train.csv")
test_data = pd.read_csv("/kaggle/input/credibility-data/full_data_test.csv")
dev_data = pd.read_csv("/kaggle/input/credibility-data/full_data_dev.csv")

train_data.head()

,title,text,label
0,'protests in paris ahead of putin visit to fre...,'paris (ap) - human rights activists are ga...,1
1,"'donald trump gives $10,000 to pastor's family'",'chilling: what netanyahu is bracing for obama...,0
2,'california democrats propose in-state tuition...,'california democrats have proposed a law to g...,1
3,'inner earth glows like in the movie avatar','can there be light below the surface of the e...,0
4,'clinton foundation ceo goes missing after tru...,' another astonishing security council (sc) re...,0


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
import torch

In [4]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# create concatenated df
train_df = train_data
train_df["text"] = tokenizer.cls_token + train_df['title'] + tokenizer.sep_token + train_df['text'] + tokenizer.sep_token
train_df = train_df.drop(['title'], axis=1)

dev_df = dev_data
dev_df["text"] = tokenizer.cls_token + dev_df['title'] + tokenizer.sep_token + dev_df['text'] + tokenizer.sep_token
dev_df = dev_df.drop(['title'], axis=1)

test_df = test_data
test_df["text"] = tokenizer.cls_token + test_df['title'] + tokenizer.sep_token + test_df['text'] + tokenizer.sep_token
test_df = test_df.drop(['title'], axis=1)

train_df.head()

,text,label
0,[CLS]'protests in paris ahead of putin visit t...,1
1,"[CLS]'donald trump gives $10,000 to pastor's f...",0
2,[CLS]'california democrats propose in-state tu...,1
3,[CLS]'inner earth glows like in the movie avat...,0
4,[CLS]'clinton foundation ceo goes missing afte...,0


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Calculate metrics using scikit-learn
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [13]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        label = self.data.iloc[idx]['label']

        # Tokenize text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }
train_dataset = CustomDataset(train_df, tokenizer, 512)
dev_dataset = CustomDataset(dev_df, tokenizer, 512)

In [14]:
from transformers import Trainer, TrainingArguments
import os

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='/kaggle/working/models',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/kaggle/working/logs',            # directory for storing
    logging_steps=10,                # log training loss every n steps
    evaluation_strategy="epoch",     # evaluate model at the end of each epoch
    save_strategy="epoch",             # save model checkpoint at the end of each epoch
    save_total_limit=3,              # Limit the total number of saved models
    save_steps=500,
)

credibility_trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments
    train_dataset=train_dataset,     # training dataset
    eval_dataset=dev_dataset,        # evaluation dataset
    tokenizer=tokenizer,             # tokenizer for encoding input data
    compute_metrics=get_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
print(train_df.shape)
print(dev_df.shape)

text = tokenizer.cls_token + "this needs to be tokenized" + tokenizer.sep_token
tokenizer.encode(text)

(15726, 2)
(1966, 2)


[101, 101, 2023, 3791, 2000, 2022, 19204, 3550, 102, 102]

In [16]:
credibility_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.039000,0.024327,0.988301,0.988596,0.988301,0.988312
2,0.021900,0.019157,0.992370,0.992379,0.992370,0.992368
3,0.045700,0.025318,0.992370,0.992387,0.992370,0.992372


TrainOutput(global_step=2949, training_loss=0.0539198517416853, metrics={'train_runtime': 3190.3896, 'train_samples_per_second': 14.788, 'train_steps_per_second': 0.924, 'total_flos': 6249546933792768.0, 'train_loss': 0.0539198517416853, 'epoch': 3.0})

In [13]:
# save model
model_path = "/kaggle/working/results"
os.makedirs(model_path, exist_ok=True)

model_file = os.path.join(model_path, "model.pth")
torch.save(model.state_dict(), model_file)

tokenizer_file = os.path.join(model_path, "tokenizer.json")
tokenizer.save_pretrained(model_path)

('/kaggle/working/results/tokenizer_config.json',
 '/kaggle/working/results/special_tokens_map.json',
 '/kaggle/working/results/vocab.txt',
 '/kaggle/working/results/added_tokens.json')

In [15]:
# load model
bert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/results/model.pth")
pretrained_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/results/")

OSError: Incorrect path_or_model_id: '/kaggle/working/results/model.pth'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [19]:
output_dir = '/kaggle/working/results'
text_content = "This is a test file."

# Create a new text file and write content into it
with open(output_dir + '/test_file.txt', 'w') as f:
    f.write(text_content)
    

In [23]:
if os.path.exists('/kaggle/working/results/model.pth'):
    print("Model file exists.")
else:
    print("Model file does not exist.")
    
model_checkpoint = "/kaggle/working/models/checkpoint-1.pth"

if os.path.exists(model_checkpoint):
    print("Model file exists.")
else:
    print("Model file does not exist.")
    
import os

# Define the directory path
directory = '/kaggle/working/models/checkpoint-2949'

# List the files in the directory
files = os.listdir(directory)

# Print the list of files
print("Directory contents:")
for file in files:
    print(file)
    


Model file exists.
Model file does not exist.
Directory contents:
tokenizer_config.json
trainer_state.json
training_args.bin
model.safetensors
vocab.txt
special_tokens_map.json
optimizer.pt
scheduler.pt
config.json
rng_state.pth


In [22]:
import shutil
import os

# Path to the checkpoint file
checkpoint_path = "/kaggle/working/models/checkpoint-2949"

# Destination path for the saved checkpoint file
destination_path = "/kaggle/working/models/saved_model.pth"

# Check if the checkpoint file exists
if os.path.exists(checkpoint_path):
    # Copy the checkpoint file to the destination path
    shutil.copyfile(checkpoint_path, destination_path)
    print("Checkpoint file saved as:", destination_path)
else:
    print("Checkpoint file does not exist.")

IsADirectoryError: [Errno 21] Is a directory: '/kaggle/working/models/checkpoint-2949'

In [20]:
from IPython.display import FileLink

# Replace 'model_file.bin' with the actual name of your model file
model_file_path = '/kaggle/working/results/model.pth'

# Create a link to download the model file
FileLink(model_file_path)

/kaggle/working/results/model.pth

In [17]:
test_dataset = CustomDataset(test_df, tokenizer, 512)

results = credibility_trainer.evaluate(eval_dataset=test_dataset)

results_df = pd.DataFrame(results, index=[0])
results_df

,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.037339,0.991349,0.991355,0.991349,0.99135,87.1101,22.558,0.356,3.0


In [28]:
# Recreate the model architecture

# Load the trained weights from the checkpoint file
checkpoint_path = "/kaggle/working/models/checkpoint-2949"
#model.load_state_dict(torch.load(os.path.join(checkpoint_path, "model.safetensors")))
model = DistilBertForSequenceClassification.from_pretrained(checkpoint_path)

In [31]:
# Instantiate Trainer with the trained model and evaluation arguments
trainer = Trainer(
    model=model,  # The trained model
    tokenizer=tokenizer,  # The tokenizer associated with the model
    compute_metrics=get_metrics,  # Function to compute evaluation metrics
)

# Evaluate the model
results = trainer.evaluate(test_dataset)

# Print the evaluation results
results_df = pd.DataFrame(results, index=[0])
results_df

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.037339,0.991349,0.991355,0.991349,0.99135,86.2748,22.776,2.851


In [34]:
# Function to get predicted probabilities for all elements in the dataset
def get_predicted_probabilities(dataset):
    # Initialize list to store predicted probabilities
    all_predicted_probabilities = []
    
    # Iterate over each element in the dataset
    for i in range(len(dataset)):
        # Get the input_ids and attention_mask from the dataset
        input_ids = dataset[i]['input_ids']
        attention_mask = dataset[i]['attention_mask']
        
        # Perform inference
        with torch.no_grad():
            outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        
        # Get predicted probabilities
        predicted_probabilities = torch.softmax(outputs.logits, dim=1)
        
        # Append predicted probabilities to the list
        all_predicted_probabilities.append(predicted_probabilities.numpy()[0].tolist())
    
    return all_predicted_probabilities

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Get predicted probabilities for all elements in the dataset
predicted_probabilities = get_predicted_probabilities(test_dataset)

# Print the predicted probabilities
print(predicted_probabilities)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [35]:
import shutil

# Define the directory path
directory_path = '/kaggle/working/models/checkpoint-2949'

# Define the output zip file path
zip_file_path = '/kaggle/working/models/checkpoint-2949.zip'

# Create a zip file containing the contents of the directory
shutil.make_archive(zip_file_path.split('.zip')[0], 'zip', directory_path)

# Provide a download link to the zip file
print(f'Download the zip file containing the directory contents: [{zip_file_path}]({zip_file_path})')

Download the zip file containing the directory contents: [/kaggle/working/models/checkpoint-2949.zip](/kaggle/working/models/checkpoint-2949.zip)


In [36]:
from IPython.display import FileLink

# Provide a download link to the zip file
FileLink("/kaggle/working/models/checkpoint-2949.zip")

/kaggle/working/models/checkpoint-2949.zip

In [37]:
from IPython.display import HTML

# Create a download link
HTML(f'<a href="/kaggle/working/models/checkpoint-2949.zip" download>Click here to download the zip file</a>')

In [38]:
import os
import shutil

# Define the directory path
directory = '/kaggle/working/models/checkpoint-2949'

# Define the output zip file path
zip_file_path = '/kaggle/working/checkpoint-2949.zip'

# Create a zip file of the directory contents
shutil.make_archive('/kaggle/working/checkpoint-2949', 'zip', directory)

# Create a download link
HTML(f'<a href="{zip_file_path}" download>Click here to download the zip file</a>')

In [39]:
if os.path.exists(zip_file_path):
    print("Model file exists.")
else:
    print("Model file does not exist.")

Model file exists.


In [40]:
FileLink(zip_file_path)


/kaggle/working/checkpoint-2949.zip

In [41]:
FileLink(zip_file_path)


/kaggle/working/checkpoint-2949.zip